In [ ]:
Слишком плохие эмбединги
Не уверен все ли верно считается

In [2]:
import numpy as np
from tqdm import tqdm
import random
# f = open('/Users/georgychernousov/Downloads/example.txt','r')
f = open('reviews.txt','r') # What we know!
reviews = f.readlines()
f.close()

# tokenize
tokens = ' '.join(reviews).split()[:1000000]

# __init__
alpha = 0.1
iterations = 2
hidden_size = 50
window = 3
negative = 5
vocab = list(set(tokens))
weights = (np.random.rand(len(vocab),hidden_size) - 0.5) * 0.2
weights_1 = np.zeros([len(vocab), hidden_size])
sigmoid = lambda x: 1/(1 + np.exp(-x))
l_2_target = np.zeros(negative+1)
l_2_target[0] = 1

for i in range(iterations):
    c = 0
    for word_i in tqdm(range(len(tokens))):
        target_word = vocab.index(tokens[word_i])
        target_samples = [target_word, *[random.randint(0,len(vocab)-1) for i in range(negative)]]
        # print(target_samples)

        left_context = tokens[max(0,target_word-window):target_word]
        right_context = tokens[target_word+1:min(len(tokens),target_word+window+1)]

        indexes = [*list(range(max(0,target_word-window), target_word)), *list(range(target_word+1, min(len(vocab)-1,target_word+window+1)))]
        l_1 = np.mean(weights[indexes],axis=0)
        l_2 = sigmoid(l_1.dot(weights_1[target_samples].T))

        # видимо что то типа backprop
        l_2_delta = l_2 - l_2_target # loss
        l_1_delta = l_2_delta.dot(weights_1[target_samples])  # типа градиент

        weights[indexes] -= l_1_delta * alpha
        weights_1[target_samples] -= np.outer(l_2_delta,l_1)*alpha


    # break
# train(reviews[0])

100%|██████████| 1000000/1000000 [1:29:42<00:00, 185.78it/s]  


In [6]:
from sentence_transformers.util import cos_sim
x = [cos_sim(weights[vocab.index('human'),:], weights[i,:]).numpy()[0][0] for i in tqdm(range(weights.shape[0]))]
{k: v for k, v in sorted(dict(zip(vocab, x)).items(), key=lambda item: item[1], reverse=True)}

100%|██████████| 32424/32424 [00:51<00:00, 633.94it/s]


{'human': 1.0,
 'rod': 0.8813802271362094,
 'the': 0.880166616443343,
 'warfare': 0.8734203579412314,
 'innocents': 0.8719290638339152,
 'fewer': 0.86680137735762,
 'insistence': 0.866259590662644,
 'cell': 0.8654300701350433,
 'decided': 0.8646384858582936,
 'them': 0.8645121889673859,
 'innocence': 0.8639388835461723,
 'bender': 0.8636733945467936,
 'young': 0.8611056927912796,
 'like': 0.8609251927104494,
 'bootleg': 0.8605350373409797,
 'pure': 0.8584752949791608,
 'extremely': 0.8568128246340434,
 'those': 0.8554862570568469,
 'starts': 0.8554019570352351,
 'actors': 0.8540382445454475,
 'budgets': 0.8540378002875176,
 'should': 0.8524347241107226,
 'your': 0.8512074716214337,
 'pretty': 0.850724110295748,
 'that': 0.8505750646919322,
 'not': 0.8493238593323116,
 'into': 0.8491363434463886,
 'oh': 0.8485117475242483,
 'own': 0.8465788519541769,
 'mantegna': 0.8440625541408303,
 'done': 0.8435081602990125,
 'act': 0.8432948383189921,
 'be': 0.8428333463343864,
 'they': 0.8403526981